In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
survey_df = pd.read_csv('all_apps_wide_2025-03-22.csv')
#excluding test runs from bilendi
#exclude nan and test run
survey_df = survey_df[(survey_df['participant.label'].notna()) & (~survey_df['participant.label'].isin(['testthibaut', '123456', 'test1']))]
# filter screenout and quota full
#survey_df = survey_df[survey_df['participant.screen_out'] != 1]
#survey_df = survey_df[survey_df['participant.quota']==0]

/var/folders/_2/9qtg_rwn3k78ky0gds0t023m0000gn/T/ipykernel_9184/4176319859.py:1: DtypeWarning: Columns (2,622,624) have mixed types. Specify dtype option on import or set low_memory=False.
  survey_df = pd.read_csv('all_apps_wide_2025-03-22.csv')


In [3]:
survey_df

,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,...,EndApp.1.player.payoff,EndApp.1.player.left_right_ideology,EndApp.1.player.edu_general_education,EndApp.1.player.edu_general_education_open,EndApp.1.player.edu_vocational_training,EndApp.1.player.edu_vocational_training_open,EndApp.1.player.income_net_income,EndApp.1.player.time_end,EndApp.1.group.id_in_subsession,EndApp.1.subsession.round_number
3,4,pm7ou0sk,301699035802866,0,3,131,SocialmediaApp,PreQuestionsPage,2025-03-17 11:10:58.698515,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-999,1,1
4,5,d8bxmrvr,301699035810036,0,131,131,EndApp,Redirect,2025-03-17 11:11:36.907847,1,...,0.0,3.0,4.0,NaN,5.0,NaN,10.0,2025-03-17T11:23:29.868Z,1,1
8,9,wxs32cf6,303396094903540,0,130,131,EndApp,End,2025-03-17 16:04:36.876541,1,...,0.0,5.0,3.0,NaN,3.0,NaN,7.0,-999,1,1
9,10,dtewqefp,303396094630938,0,130,131,EndApp,End,2025-03-17 16:05:06.423210,1,...,0.0,3.0,2.0,NaN,2.0,NaN,1.0,-999,1,1
10,11,p647yt6k,303396094668864,0,6,131,SocialmediaApp,ContentPage,2025-03-17 16:07:46.943418,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-999,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1496,ab3yebpc,303723786595546.0,0,3,131,SocialmediaApp,PreQuestionsPage,2025-03-22 08:38:59.637371,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-999,1,1
1496,1497,a5jelp3v,303639031292062.0,0,131,131,EndApp,Redirect,2025-03-22 08:41:35.123762,1,...,0.0,5.0,4.0,NaN,0.0,NaN,5.0,2025-03-22T08:46:05.132Z,1,1
1497,1498,lvwxbige,303483423714405.0,0,3,131,SocialmediaApp,PreQuestionsPage,2025-03-22 08:49:32.410971,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-999,1,1
1498,1499,14mq5z8i,303639033807896.0,0,6,131,SocialmediaApp,ContentPage,2025-03-22 08:56:45.696451,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-999,1,1


In [4]:
#### server went down #####
###########################

survey_df[survey_df['participant._current_page_name']=='Screening']

survey_df['participant.time_started_utc'] = pd.to_datetime(survey_df['participant.time_started_utc'])
filtered_df = survey_df[survey_df['participant._current_page_name']=='Screening']
start_time = pd.Timestamp('2025-03-19 11:38:14.711290')
end_time = pd.Timestamp('2025-03-19 13:13:22.021951')

# Apply the filter
filtered_df = filtered_df[(filtered_df['participant.time_started_utc'] > start_time) & 
                 (filtered_df['participant.time_started_utc'] < end_time)]

affected_ids = filtered_df[["participant.time_started_utc", "participant.label"]]
#affected_ids
#affected_ids.to_csv('affected_participants_labels.csv', index=False) 

In [5]:
# check quota full
#survey_df[survey_df['participant.quota']==1]

In [6]:
def get_stats (Start_Quest_dt):
    #taking those who at the End_Quest app name
    End_Quest_app_dt = Start_Quest_dt[Start_Quest_dt['participant._current_app_name']=='EndApp']
    #and taking only those who either finished and submitted or just finished
    Completed_Survey = End_Quest_app_dt[(End_Quest_app_dt['participant._current_page_name']=='Redirect') | (End_Quest_app_dt['participant._current_page_name']=='Last_Page') | (End_Quest_app_dt['participant._current_page_name']=='End')]
    # Complete ids
    complete_ids = Completed_Survey['participant.label']
    #Quota
    #age
    Age_group = Completed_Survey[['participant.label','participant.age_group']].groupby('participant.age_group').count()
    Age_group.rename(columns= {'participant.label':'Quota'}, inplace = True)
    # Define the bins and labels for the groups
    
    labels = ['18-29', '30-39', '40-49', '50-59', '60-69']
    bins = list(range(-1, -1 + len(labels)+1))
    # Create a new column 'Age_Groups' based on the bins
    Age_group['Age_Groups'] = pd.cut(Age_group.index, bins=bins, labels=labels, include_lowest=True)
    Age_group.set_index('Age_Groups', drop=True, inplace=True)
    #gender
    Gender_group = Completed_Survey[['participant.label','participant.gender']].groupby('participant.gender').count()
    Gender_group.rename(columns= {'participant.label':'Quota'}, inplace = True)
    # Define the bins and labels for the groups

    labels = ['female', 'male', 'other', 'anonymous']
    bins = list(range(-1, -1 + len(labels)+1))
    # Create a new column 'Age_Groups' based on the bins
    Gender_group['Gender_Groups'] = pd.cut(Gender_group.index, bins=bins, labels=labels, include_lowest=True)
    Gender_group.set_index('Gender_Groups', drop=True, inplace=True)
    
    #state
    State_group = Completed_Survey[['participant.label','participant.federal_state']].groupby('participant.federal_state').count()
    State_group.rename(columns= {'participant.label':'Quota'}, inplace = True)
    # Define the bins and labels for the groups
    labels = ["Schleswig-Holstein",
            "Hamburg",
            "Niedersachsen",
            "Bremen",
            "Nordrhein-Westfalen",
            "Hessen",
            "Rheinland-Pfalz",
            "Baden-Württemberg",
            "Bayern",
            "Saarland",
            "Berlin",
            "Brandenburg",
            "Mecklenburg-Vorpommern",
            "Sachsen",
            "Sachsen-Anhalt",
            "Thüringen",
            "Weiß nicht"]
    bins = list(range(-1, -1 + len(labels)+1))
    # Create a new column 'Age_Groups' based on the bins
    State_group['State_Groups'] = pd.cut(State_group.index, bins=bins, labels=labels, include_lowest=True)
    
    State_group.set_index('State_Groups', drop=True, inplace=True)
    
    total_participants = len(Start_Quest_dt)
    completed_participants = len(Completed_Survey)
    dropout_rate = (total_participants - completed_participants) / total_participants * 100
    
    print('We have {} out of {} participants, who finished the survey'.format(len(Completed_Survey), len(Start_Quest_dt)))
    print('The average dropout rate is {}'.format(dropout_rate))
    print('__________________')
    print('Current Quota:')
    print(Age_group)
    print(Gender_group)
    print(State_group)
    complete_ids.to_csv('complete_participants_labels.csv', index=False) 

In [7]:
get_stats(survey_df)

We have 1032 out of 1489 participants, who finished the survey
The average dropout rate is 30.691739422431162
__________________
Current Quota:
            Quota
Age_Groups       
18-29         184
30-39         201
40-49         192
50-59         243
60-69         212
               Quota
Gender_Groups       
female           504
male             522
other              5
anonymous          1
                        Quota
State_Groups                 
Schleswig-Holstein         25
Hamburg                    21
Niedersachsen             105
Bremen                     10
Nordrhein-Westfalen       220
Hessen                     81
Rheinland-Pfalz            54
Baden-Württemberg         136
Bayern                    168
Saarland                   11
Berlin                     46
Brandenburg                33
Mecklenburg-Vorpommern     21
Sachsen                    51
Sachsen-Anhalt             28
Thüringen                  22


In [8]:
def dropout_rate_per_day (survey_data):
    
    survey_data['participant.time_started_utc'] = pd.to_datetime(survey_data['participant.time_started_utc'])

    # Extract the day from the datetime
    survey_data['day_started'] = survey_data['participant.time_started_utc'].dt.date

    # Count the total number of participants who started on each day
    total_participants_per_day = survey_data['day_started'].value_counts().sort_index()
    # Filter only participants who finished the survey
    finished_dt = survey_data[survey_data['participant._current_app_name']=='EndApp']
    #and taking only those who either finished and submitted or just finished
    finished_dt = finished_dt[(finished_dt['participant._current_page_name']=='Redirect') | (finished_dt['participant._current_page_name']=='Last_Page') | (finished_dt['participant._current_page_name']=='End')]

    finished_participants_per_day = finished_dt['day_started'].value_counts().sort_index()

    # Calculate dropout rate for each day
    dropout_rate_per_day = 100 - ((finished_participants_per_day * 100) / total_participants_per_day)

    # Display the result
    result_df = pd.DataFrame({
        'Total Participants': total_participants_per_day,
        'Dropout Rate': dropout_rate_per_day
    })

    print(result_df)

In [9]:
dropout_rate_per_day(survey_df)

             Total Participants  Dropout Rate
day_started                                  
2025-03-17                   27     22.222222
2025-03-18                  185     14.594595
2025-03-19                  416     25.000000
2025-03-20                  523     23.518164
2025-03-21                  313     56.230032
2025-03-22                   25     84.000000


In [10]:
# List of original columns to keep
# quota variables from the database

columns_to_keep = [
    'session.completed_gender_0',
    'session.completed_gender_1',
    'session.completed_gender_2',
    'session.completed_gender_3',
    'session.completed_age_group_0',
    'session.completed_age_group_1',
    'session.completed_age_group_2',
    'session.completed_age_group_3',
    'session.completed_age_group_4',
    'session.completed_federal_state_0',
    'session.completed_federal_state_1',
    'session.completed_federal_state_2',
    'session.completed_federal_state_3',
    'session.completed_federal_state_4',
    'session.completed_federal_state_5',
    'session.completed_federal_state_6',
    'session.completed_federal_state_7',
    'session.completed_federal_state_8',
    'session.completed_federal_state_9',
    'session.completed_federal_state_10',
    'session.completed_federal_state_11',
    'session.completed_federal_state_12',
    'session.completed_federal_state_13',
    'session.completed_federal_state_14',
    'session.completed_federal_state_15'
]

# Corresponding new column names
new_column_names = [
    'female',
    'male',
    'other',
    "anonymous",
    '[18,29]',
    '[30,39]',
    '[40,49]',
    '[50,59]',
    '[60,69]',
    "Schleswig-Holstein",
    "Hamburg",
    "Niedersachsen",
    "Bremen",
    "Nordrhein-Westfalen",
    "Hessen",
    "Rheinland-Pfalz",
    "Baden-Württemberg",
    "Bayern",
    "Saarland",
    "Berlin",
    "Brandenburg",
    "Mecklenburg-Vorpommern",
    "Sachsen",
    "Sachsen-Anhalt",
    "Thüringen"
]

# Filter and rename columns
survey_df = survey_df[columns_to_keep].rename(columns=dict(zip(columns_to_keep, new_column_names)))
df1 = survey_df[:1].T
df1.rename(columns={3: "count"}, inplace=True)
df1

,count
female,444
male,442
other,5
anonymous,2
"[18,29]",170
"[30,39]",176
"[40,49]",164
"[50,59]",208
"[60,69]",175
Schleswig-Holstein,23


#### Below we needed to reset the quota numbers and recalculate the quota due to some issues

In [11]:
#the difference that we are missing and we should add to quota count

data = {
    "category": [
        "female", "male", "other", "anonymous",
        "[18,29]", "[30,39]", "[40,49]", "[50,59]", "[60,69]",
        "Schleswig-Holstein", "Hamburg", "Niedersachsen", "Bremen", "Nordrhein-Westfalen",
        "Hessen", "Rheinland-Pfalz", "Baden-Württemberg", "Bayern", "Saarland", "Berlin",
        "Brandenburg", "Mecklenburg-Vorpommern", "Sachsen", "Sachsen-Anhalt", "Thüringen"
    ],
    "count": [
        61, 80, 0, 0,
        16, 25, 28, 35, 37,
        3, 4, 13, 0, 34,
        12, 4, 20, 18,
        1, 4, 4, 7,
        7, 3, 7
    ]
}

df = pd.DataFrame(data).set_index("category")
df

,count
category,
female,61
male,80
other,0
anonymous,0
"[18,29]",16
"[30,39]",25
"[40,49]",28
"[50,59]",35
"[60,69]",37


In [12]:
#final quota (real numbers)
df_combined = pd.concat([df, df1])
df_summed = df_combined.groupby(level=0).sum()
df_summed

,count
Baden-Württemberg,136
Bayern,168
Berlin,46
Brandenburg,33
Bremen,10
Hamburg,21
Hessen,81
Mecklenburg-Vorpommern,21
Niedersachsen,105
Nordrhein-Westfalen,221
